<pre>
The 10 classes to predict are:

c0: safe driving
c1: texting - right
c2: talking on the phone - right
c3: texting - left
c4: talking on the phone - left
c5: operating the radio
c6: drinking
c7: reaching behind
c8: hair and makeup
c9: talking to passenger
</pre>

In [1]:
# If you want to extract the features locally, run:
# !python train_extract.resnet50.py
# !python test_extract.resnet50.py

In [2]:
# If you want to train your own gl model, run:
!python train_gl_model.py

This non-commercial license of GraphLab Create for academic use is assigned to jekyll4168@icloud.com and will expire on August 14, 2017.
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

Boosted trees classifier:
--------------------------------------------------------
Number of examples          : 21266
Number of classes           : 10
Number of feature columns   : 1
Number of unpacked features : 2048
+-----------+--------------+-------------------+---------------------+-------------------+---------------------+
| Iteration | Elapsed Time | Training-accuracy | Validation-accuracy | Training-log_loss | Validation-log_loss |
+-----------+--------------+-------------------+---------------------+-------------------+---------------------+
| 1         | 12.212171    | 0.543873          | 0.531088            | 2.162477          | 2.163339            |
| 2         | 23.2851

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\JIANCH~1\AppData\Local\Temp\graphlab_server_1471189330.log.0


In [3]:
from utils import train_set, test_set, submit
from gl_utils import get_sf, load_model, result_sf2df

%load_ext autoreload
%autoreload 2

In [4]:
train_sf = get_sf(train_set, times=1)
test_sf = get_sf(test_set, times=5)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\JIANCH~1\AppData\Local\Temp\graphlab_server_1471195774.log.0


This non-commercial license of GraphLab Create for academic use is assigned to jekyll4168@icloud.com and will expire on August 14, 2017.


In [5]:
train_sf.head(3)

classname,subject,path,img,conv
c0,p002,data/imgs/train/c0/img_44733.jpg ...,img_44733.jpg,"[0.400213897228,0.637386381626, ..."
c0,p002,data/imgs/train/c0/img_72999.jpg ...,img_72999.jpg,"[0.417709350586,0.587353050709, ..."
c0,p002,data/imgs/train/c0/img_25094.jpg ...,img_25094.jpg,"[0.926794111729,0.720134198666, ..."


In [6]:
test_sf.head(3)

img,path,conv
img_69813.jpg,data/imgs/test/img_69813.jpg ...,"[0.66599714756,0.219444215298, ..."
img_33495.jpg,data/imgs/test/img_33495.jpg ...,"[0.617196261883,0.394609481096, ..."
img_47565.jpg,data/imgs/test/img_47565.jpg ...,"[1.40946865082,1.53018343449, ..."


In [7]:
boosting = load_model()
# import graphlab as gl
# boosting = gl.boosted_trees_classifier.create(dataset=train_sf, target='classname', features=['conv'],
#                                               resume_from_checkpoint='.gl_checkpoints/boosting/model_checkpoint_400')
boosting

Class                          : BoostedTreesClassifier

Schema
------
Number of examples             : 21266
Number of feature columns      : 1
Number of unpacked features    : 2048
Number of classes              : 10

Settings
--------
Number of trees                : 6000
Max tree depth                 : 3
Training time (sec)            : 6410.2808
Training accuracy              : 1.0
Validation accuracy            : 0.9914
Training log_loss              : 0.0016
Validation log_loss            : 0.0338

In [8]:
train_df = result_sf2df(boosting, train_sf)
test_df = result_sf2df(boosting, test_sf)

In [9]:
train_df.head(3)

,classname,subject,path,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,c0,p002,data/imgs/train/c0/img_44733.jpg,img_44733.jpg,0.991858,0.000263,0.002836,0.000546,0.002031,0.000055,0.000317,0.000049,0.000243,0.001801
1,c0,p002,data/imgs/train/c0/img_72999.jpg,img_72999.jpg,0.990238,0.000107,0.002033,0.001292,0.001827,0.000091,0.000153,0.000037,0.000145,0.004078
2,c0,p002,data/imgs/train/c0/img_25094.jpg,img_25094.jpg,0.999389,0.000024,0.000033,0.000006,0.000182,0.000002,0.000007,0.000002,0.000011,0.000344


In [10]:
test_df.head(3)

,img,path,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_69813.jpg,data/imgs/test/img_69813.jpg,0.000093,0.000004,0.000908,0.000117,0.995179,0.000083,0.000582,0.001137,0.001632,0.000264
1,img_33495.jpg,data/imgs/test/img_33495.jpg,0.001184,0.000268,0.000004,0.000050,0.000082,0.996905,0.000012,0.000030,0.001262,0.000205
2,img_47565.jpg,data/imgs/test/img_47565.jpg,0.004076,0.001038,0.044172,0.001867,0.874826,0.000708,0.002522,0.033358,0.021313,0.016120


In [11]:
import pandas as pd
info = pd.concat([submit(train_df,include_path=True,filename=None,include_class=True),
                  submit(test_df,include_path=True,filename=None,include_class=True)])
info['path'] = info['path'].map(lambda path: '/'.join(path.split('/')[1:]))
info.to_csv('data/imgs_proba.csv')

No "classname" column!


In [12]:
submit(test_df,filename="6000.csv")

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img,,,,,,,,,,
img_69813.jpg,0.000093,4.250471e-06,9.077415e-04,0.000117,0.995179,8.326469e-05,0.000582,1.137299e-03,0.001632,0.000264
img_33495.jpg,0.001184,2.681123e-04,3.576040e-06,0.000050,0.000082,9.969048e-01,0.000012,2.967211e-05,0.001262,0.000205
img_47565.jpg,0.004076,1.038156e-03,4.417210e-02,0.001867,0.874826,7.078826e-04,0.002522,3.335797e-02,0.021313,0.016120
img_19469.jpg,0.000125,1.973810e-04,1.159406e-05,0.000007,0.000394,1.622221e-05,0.000064,2.043488e-04,0.998269,0.000711
img_70575.jpg,0.976370,3.910580e-04,1.030223e-03,0.000729,0.000726,2.871113e-04,0.000063,6.221078e-05,0.015445,0.004895
img_73511.jpg,0.000245,1.221677e-04,1.014076e-04,0.000128,0.000197,5.474058e-05,0.000004,9.966270e-01,0.002261,0.000259
img_8786.jpg,0.074186,8.621611e-02,4.035215e-04,0.228493,0.254649,7.201182e-03,0.000827,2.244411e-03,0.328916,0.016865
img_88601.jpg,0.017340,1.823108e-01,2.217649e-03,0.004226,0.005665,9.750558e-04,0.753398,1.742933e-02,0.003460,0.012978
img_92431.jpg,0.307442,3.294233e-03,1.236095e-02,0.001270,0.000050,8.357598e-03,0.001651,6.256204e-01,0.006360,0.033594


In [ ]:
# !sudo pip install kaggle-cli
# !kg config -g -u <username> -p <password>
# !kg submit result/150.csv -c state-farm-distracted-driver-detection -m "Naive solution with pretrained ResNet-50 and 150 boosting trees"